In [19]:
#20221212修改
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import time


from pylab import rcParams
import matplotlib.cm as cm
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder  # 编码转换
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier  # 随机森林
from sklearn.svm import SVC, LinearSVC  # 支持向量机
from sklearn.linear_model import LogisticRegression  # 逻辑回归
from sklearn.neighbors import KNeighborsClassifier  # KNN算法
from sklearn.naive_bayes import GaussianNB  # 朴素贝叶斯
from sklearn.tree import DecisionTreeClassifier  # 决策树分类器
# from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score,accuracy_score,roc_auc_score
from sklearn.metrics import confusion_matrix
#import sys 
#print(sys.path)
print("当前路径 -> %s" %os.getcwd())

import warnings
warnings.filterwarnings("ignore")

class Nstr:
    def __init__(self,arg):
        self.x=arg
    def __sub__(self,other):
        c=self.x.replace(other.x,"")
        return c

start = time.time()

##读取文件位置
filename= os.listdir(r'C:\Users\Xia HAN\Downloads\jupyter_code_data_WEI_kunni\code_20221124\ShanE_5color_Fp1z2_30_3_9\ShanE_5color_Fp1z2_30_3_9')
print(filename) ## filename为该表格的所有sheet，如'sheet1','sheet2','sheet3'

for i in range(len(filename)):  ###  5个不同颜色的 30_3_9 的文件
    
    totaldata = pd.read_excel(filename[i], sheet_name=0)
    totaldata.dropna(inplace=True)
    for j in range(0,9):                        ################################(0,9)
        print(j)
        brain = totaldata.iloc[:, 2:((j+1)*3+2)]################################### 每次读三列数据，为（Fp1 Fpz Fp2）的一组熵值
        print(brain)
        scaler = StandardScaler(copy=False)
        scaler.fit_transform(brain)
        
        brain = scaler.fit_transform(brain)
        k=100                                 ##100
        X=brain
        
        y=totaldata["group"].values
        sum_result = 0
        list_result=[]
        list_result_time=[]
        Classifiers = [
            ["Random Forest", RandomForestClassifier()],
            ["Support Vector Machine", SVC()],
            ["LogisticRegression",LogisticRegression()],
            ["KNN",KNeighborsClassifier(n_neighbors=5)],
            # ["Naive Bayes",GaussianNB()],
            ["Decision Tree",DecisionTreeClassifier()],
            # ["AdaBoostClassifier", AdaBoostClassifier()],
            # ["GradientBoostingClassifier", GradientBoostingClassifier()],
            # ["XGB", XGBClassifier()],
            ["CatBoost", CatBoostClassifier(logging_level='Silent')]
        ]
        sss = StratifiedShuffleSplit(n_splits=k, test_size=0.3, random_state=None)# n_splits：是将训练数据分成train/test对的组数
        start = time.time()                                                                           # train：test=7：3
        for train_index, test_index in sss.split(X, y):
            #print("train:", train_index, "test:", test_index)
            X_train,X_test=X[train_index], X[test_index]
            y_train,y_test=y[train_index], y[test_index]
            
            Classify_result = []
            names = []
            times = []
            prediction = []
            for name, classifier in Classifiers:   ##此时分类器类别个数为6
                start = time.time()
                classifier = classifier
                classifier.fit(X_train, y_train)
                y_pred = classifier.predict(X_test)
                acc_score = accuracy_score(y_test, y_pred)     ######################################
                class_eva = pd.DataFrame([acc_score])
                Classify_result.append(class_eva)
                name = pd.Series(name)
                names.append(name)
                end = time.time()
                exe_time = str(round((end-start),6))
                exe_time = pd.DataFrame([exe_time])
                times.append(exe_time)
                y_pred = pd.Series(y_pred)
                prediction.append(y_pred)
                ##print(Classify_result)
            names = pd.DataFrame(names)  ##这里的names是分类器的名称
            ##print(names)
            names = names[0].tolist()
            ##print(names[0])
            result = pd.concat(Classify_result, axis=1)#axis=1为列合并
            result_time=pd.concat(times,axis=1)
            ##print(result) 
            result.columns = names
            result_time.columns = names
            result.index = ["acc_score"]
            list_result.append(result)
            result_time.index = ["exe_time"]
            list_result_time.append(result_time)

            #sum_result += result
        acc_result = pd.concat(list_result, axis=0) ##axis=0为基于行合并
        #print(avg_result)
        acc_result_time = pd.concat(list_result_time, axis=0)
        
                
        fileName_xlsx = Nstr(str(filename[i]))  ##   文件名为 ShanE_xxxx_Fp1z2_30_3_9.xlsx
        fileName_suffix = Nstr(".xlsx")  
        new_fileName = fileName_xlsx - fileName_suffix   ####   文件名为 ShanE_xxxx_Fp1z2_30_3_9
        new_fileName = new_fileName+"_"+str(j+1)+"_"  ##   文件名为 ShanE_xxxx_Fp1z2_30_3_9_j
        
        acc_result.to_csv((new_fileName+"vectorLongeur"+".csv"))
        acc_result_time.to_csv((new_fileName+"exeTime"+".csv"))
        
        
        #acc_result.to_excel((new_fileName+"acc_result.xlsx"))
        #acc_result_time.to_excel((new_fileName+"acc_result_time.xlsx"))
    
        
        ##time.sleep(1)

当前路径 -> C:\Users\Xia HAN\Downloads\jupyter_code_data_WEI_kunni\code_20221124\ShanE_5color_Fp1z2_30_3_9
['ShanE_black_Fp1z2_30_3_9.xlsx']
0
    Unnamed: 2  Unnamed: 2.1  Unnamed: 2.2
0     0.898885      0.894018      0.894260
1     0.742301      0.746950      0.769478
2     0.870196      0.853185      0.870339
3     0.857283      0.857451      0.863062
4     0.915976      0.893105      0.942403
5     0.834391      0.838700      0.887496
6     0.881533      0.869239      0.880056
7     0.771341      0.795745      0.804239
8     0.856822      0.898458      0.853974
9     0.852082      0.852578      0.857693
10    0.833778      0.812627      0.812391
11    0.866284      0.855156      0.848169
12    0.858779      0.852109      0.846593
13    0.836074      0.824372      0.789664
14    0.828110      0.820882      0.809221
15    0.955910      0.947780      0.927092
16    0.905751      0.910123      0.899468
17    0.912960      0.909842      0.894175
18    0.807176      0.843137      0.847404
1

3
    Unnamed: 2  Unnamed: 2.1  Unnamed: 2.2  Unnamed: 3  Unnamed: 3.1  \
0     0.898885      0.894018      0.894260    0.917008      0.939894   
1     0.742301      0.746950      0.769478    0.918075      0.911314   
2     0.870196      0.853185      0.870339    0.926390      0.918918   
3     0.857283      0.857451      0.863062    0.847872      0.830890   
4     0.915976      0.893105      0.942403    0.882516      0.926094   
5     0.834391      0.838700      0.887496    0.878588      0.880190   
6     0.881533      0.869239      0.880056    0.879910      0.873994   
7     0.771341      0.795745      0.804239    0.781049      0.783254   
8     0.856822      0.898458      0.853974    0.857108      0.893041   
9     0.852082      0.852578      0.857693    0.880946      0.883186   
10    0.833778      0.812627      0.812391    0.844477      0.825083   
11    0.866284      0.855156      0.848169    0.855166      0.856146   
12    0.858779      0.852109      0.846593    0.834367      0.

5
    Unnamed: 2  Unnamed: 2.1  Unnamed: 2.2  Unnamed: 3  Unnamed: 3.1  \
0     0.898885      0.894018      0.894260    0.917008      0.939894   
1     0.742301      0.746950      0.769478    0.918075      0.911314   
2     0.870196      0.853185      0.870339    0.926390      0.918918   
3     0.857283      0.857451      0.863062    0.847872      0.830890   
4     0.915976      0.893105      0.942403    0.882516      0.926094   
5     0.834391      0.838700      0.887496    0.878588      0.880190   
6     0.881533      0.869239      0.880056    0.879910      0.873994   
7     0.771341      0.795745      0.804239    0.781049      0.783254   
8     0.856822      0.898458      0.853974    0.857108      0.893041   
9     0.852082      0.852578      0.857693    0.880946      0.883186   
10    0.833778      0.812627      0.812391    0.844477      0.825083   
11    0.866284      0.855156      0.848169    0.855166      0.856146   
12    0.858779      0.852109      0.846593    0.834367      0.

7
    Unnamed: 2  Unnamed: 2.1  Unnamed: 2.2  Unnamed: 3  Unnamed: 3.1  \
0     0.898885      0.894018      0.894260    0.917008      0.939894   
1     0.742301      0.746950      0.769478    0.918075      0.911314   
2     0.870196      0.853185      0.870339    0.926390      0.918918   
3     0.857283      0.857451      0.863062    0.847872      0.830890   
4     0.915976      0.893105      0.942403    0.882516      0.926094   
5     0.834391      0.838700      0.887496    0.878588      0.880190   
6     0.881533      0.869239      0.880056    0.879910      0.873994   
7     0.771341      0.795745      0.804239    0.781049      0.783254   
8     0.856822      0.898458      0.853974    0.857108      0.893041   
9     0.852082      0.852578      0.857693    0.880946      0.883186   
10    0.833778      0.812627      0.812391    0.844477      0.825083   
11    0.866284      0.855156      0.848169    0.855166      0.856146   
12    0.858779      0.852109      0.846593    0.834367      0.

8
    Unnamed: 2  Unnamed: 2.1  Unnamed: 2.2  Unnamed: 3  Unnamed: 3.1  \
0     0.898885      0.894018      0.894260    0.917008      0.939894   
1     0.742301      0.746950      0.769478    0.918075      0.911314   
2     0.870196      0.853185      0.870339    0.926390      0.918918   
3     0.857283      0.857451      0.863062    0.847872      0.830890   
4     0.915976      0.893105      0.942403    0.882516      0.926094   
5     0.834391      0.838700      0.887496    0.878588      0.880190   
6     0.881533      0.869239      0.880056    0.879910      0.873994   
7     0.771341      0.795745      0.804239    0.781049      0.783254   
8     0.856822      0.898458      0.853974    0.857108      0.893041   
9     0.852082      0.852578      0.857693    0.880946      0.883186   
10    0.833778      0.812627      0.812391    0.844477      0.825083   
11    0.866284      0.855156      0.848169    0.855166      0.856146   
12    0.858779      0.852109      0.846593    0.834367      0.